Idea below is to simulate five different players on different devices. Hopefully Tensorflow can take care of (optimising?) networking like this.

# CRT

In [3]:
from spdz import *

# SPDZ in Tensorflow

In [5]:
# def assign(x):
    
#     x0, x1 = x
    
    
    

# Simple addition of large numbers

NameError: name 'INPUT_PROVIDER' is not defined

# Linear regression

In [ ]:
    
#     res = sess.run([p], {
#         input_x: np.arange(10*10).reshape(10,10).astype(int),
#     })
# input_x, x = define((10,10))
# 
# w = variable((10,10))
# b = variable((10,10))
# 
# y_pred = add(mul(x, w), b)
# 
# # error = sub(y_pred, y)
# # gradients = 2/m * tf.matmul(tf.transpose(X), error)
# # training_op = tf.assign(theta, theta - learning_rate * gradients)
# 
# p = reveal(y_pred)

#     print(res)

# with tf.Session(config=config) as sess:
#     sess.run(tf.global_variables_initializer())
#     res = sess.run(z, values)
    
#     res = sess.run([p], {
#         input_x: np.arange(10*10).reshape(10,10).astype(int),
#     })
#     print(res)

# Dump